# Demo de Agentes con LiteLLM (Azure/OpenAI)

**Objetivo.** Tener varios agentes y llamar solo al que se necesite; además, un agente *triage* decide a cuál delegar según el idioma.  
**Enfoque.** Comentarios concisos, instrucciones claras y pasos listos para ejecutar.

---

## Requisitos rápidos

- Python 3.10+
- Paquetes: `python-dotenv` y las dependencias de tu framework (`agents`, `litellm`, etc.).  
- Archivo `.env` con:
  - `AZURE_API_KEY` (si usas Azure)
  - `AZURE_OPENAI_DEPLOYMENT_NAME` (nombre del *deployment* en Azure)

> Si usas OpenAI en lugar de Azure, ajusta el nombre del modelo/deployment según tu configuración.

## 1) Configuración del entorno

- Carga variables con `dotenv`.
- Desactiva *tracing* si no lo necesitas.
- Lee `AZURE_API_KEY` y `AZURE_OPENAI_DEPLOYMENT_NAME` desde el entorno.

## 2) Tool de ejemplo (function calling)

`get_weather(city, unit)` es una *tool* de demostración.  
- Propósito: mostrar cómo un agente puede invocar funciones.  
- En producción: conecta aquí tu API real (clima, RAG, BD, etc.).

In [1]:
from dotenv import load_dotenv
import os
from agents import Agent, Runner, set_tracing_disabled, function_tool, ModelSettings
from agents.extensions.models.litellm_model import LitellmModel

In [2]:
load_dotenv()
set_tracing_disabled(True)

API_KEY    = os.getenv("AZURE_API_KEY")
DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME")

In [3]:
@function_tool
def get_weather(city: str, unit: str = "C") -> str:
    """
    Devuelve clima falso (demo).
    """
    fake = {
        "Monterrey": {"C": "32°C, soleado", "F": "89.6°F, soleado"},
        "CDMX": {"C": "22°C, nublado", "F": "71.6°F, nublado"},
        "Madrid": {"C": "25°C, claro", "F": "77°F, claro"},
    }
    data = fake.get(city, {"C": "N/D", "F": "N/D"}).get(unit, "N/D")
    return f"Clima en {city}: {data}"

## 3) Agente en español

- Responde en español, breve y directo.  
- Puede invocar la tool `get_weather` si el usuario pide clima.

In [4]:
spanish_agent = Agent(
    name="Spanish agent",
    instructions=(
        "Responde en español, breve y directo. "
        "Si el usuario pide clima, llama a la tool get_weather."
    ),
    model=f"litellm/azure/{DEPLOYMENT}",
    tools=[get_weather],
    model_settings=ModelSettings(include_usage=True),
)

## 4) Agente en inglés

- Responde en inglés, conciso.  
- También puede usar `get_weather` si aplica.

In [5]:
english_agent = Agent(
    name="English Assistant",
    instructions=(
        "Reply in concise English. "
        "If the user requests weather, call get_weather."
    ),
    model=LitellmModel(
        model=f"azure/{DEPLOYMENT}",
        api_key=API_KEY
    ),
    tools=[get_weather],
    model_settings=ModelSettings(include_usage=True),
)

## 5) Agente *triage* (handoff por idioma)

- Detecta el idioma del mensaje.  
- **Español →** delega a *Spanish agent*.  
- **Inglés →** delega a *English Assistant*.  
- No explica la decisión; solo responde o delega.

In [6]:
triage_agent = Agent(
    name="Triage agent",
    instructions=(
        "Decide el handoff: si el mensaje está mayormente en español -> Spanish agent; "
        "si está en inglés -> English Assistant. Si dudas, responde en el idioma detectado. "
        "No expliques la decisión, solo responde o delega."
    ),
    model=f"litellm/azure/{DEPLOYMENT}",
    handoffs=[spanish_agent, english_agent],
    model_settings=ModelSettings(include_usage=True),
)

## 6) Pruebas rápidas

Ejemplos:
- Llamada al *triage* con mensaje en español e inglés.  
- Llamada directa a un agente específico.  
- Uso de la tool `get_weather` de forma implícita.

> Tip: ajusta las *prompts* de prueba a tu caso real.

In [9]:
# --- 1) Handoff automático por idioma ---
r1 = await Runner.run(triage_agent, input="¿Me dices el clima de Monterrey en C?")
print("\n[TRIAGE -> AUTO] (español):")
print(r1.final_output)

r2 = await Runner.run(triage_agent, input="What's the weather in Madrid in F?")
print("\n[TRIAGE -> AUTO] (english):")
print(r2.final_output)

# --- 2) Llamada directa a un solo agente ---
r3 = await Runner.run(spanish_agent, input="Dame un resumen en 2 líneas de RAG.")
print("\n[DIRECT -> Spanish agent]:")
print(r3.final_output)

# --- 3) Tool explícita (demostración) ---
r4 = await Runner.run(english_agent, input="Get me the weather for CDMX in C.")
print("\n[DIRECT -> English Assistant + tool]:")
print(r4.final_output)


c:\Users\yahir.ponce\OneDrive - Softtek\Documents\multiagent\.venv\Lib\site-packages\pydantic\_internal\_typing_extra.py:378: RuntimeWarning: coroutine 'AgentRunner.run' was never awaited
  return eval_type_backport(value, globalns, localns), True



[TRIAGE -> AUTO] (español):
El clima en Monterrey es de 32°C y soleado.

[TRIAGE -> AUTO] (english):
The weather in Madrid is 77°F and clear.

[DIRECT -> Spanish agent]:
RAG (Retrieval-Augmented Generation) es una técnica que combina la generación de texto con la recuperación de información, permitiendo a los modelos acceder a una base de datos externa para mejorar la calidad y precisión de las respuestas. Este enfoque es útil en tareas como la respuesta a preguntas y la generación de contenido informativo.

[DIRECT -> English Assistant + tool]:
The weather in CDMX is 22°C and cloudy.
